In [115]:
import pandas as pd

In [116]:
test_data = pd.read_excel('data/Список товаров.xlsx')
category_tree = pd.read_excel('data/Дерево категорий.xlsx')


In [117]:
train_data_nf = pd.read_excel('data/Данные поставщика.xlsx')

In [118]:
train_data = train_data_nf[['Название','Раздел']]

In [119]:
train_data = train_data.rename(columns={'Название': 'Name','Раздел': 'Section'})
category_tree = category_tree.rename(columns={'Главная категория': 'Main_category', 'Дочерняя категория': 'Child_category', 'Тип товара': 'Product_type'})
test_data = test_data.rename(columns={'Наименование':'Name', 'Тип товара':'Product_type'})

In [120]:
def get_category_part_by_index(cat_path: str, index: int) -> str:
    if not isinstance(index, int):
        raise TypeError(f'Index must be integer but was {type(index)}')
    if 0 <= index < 3:
        splitted_cat_path = cat_path.split('/')
        if len(splitted_cat_path) == 2:
            if index == 1:
                return ''
            if index == 2:
                return splitted_cat_path[-1]
        return splitted_cat_path[index]
    raise IndexError(f'Index was out of range')

In [156]:
train_data

,Name,Section,Main_category,Child_category,Product_type
0,"Бахилы ПНД, 50 пар// Россия",Отделочный инструмент/Средства индивидуальной ...,Отделочный инструмент,Средства индивидуальной защиты,Бахилы
1,"Комбинезон защитный СТАНДАРТ, 40 г/м2, для ма...",Отделочный инструмент/Средства индивидуальной ...,Отделочный инструмент,Средства индивидуальной защиты,Комбинезоны защитные
2,"Комбинезон защитный СТАНДАРТ, 40 г/м2, для ма...",Отделочный инструмент/Средства индивидуальной ...,Отделочный инструмент,Средства индивидуальной защиты,Комбинезоны защитные
3,"Комбинезон защитный ЛЮКС, 60 г/м2, для малярны...",Отделочный инструмент/Средства индивидуальной ...,Отделочный инструмент,Средства индивидуальной защиты,Комбинезоны защитные
4,"Комбинезон защитный ЛЮКС, 60 г/м2, для малярны...",Отделочный инструмент/Средства индивидуальной ...,Отделочный инструмент,Средства индивидуальной защиты,Комбинезоны защитные
...,...,...,...,...,...
14428,"Генератор инверторный GS-3000iS, 3,0 кВт, 230 ...",Силовое оборудование/Генераторы/Генераторы инв...,Силовое оборудование,Генераторы,Генераторы инверторные
14429,"Пила цепная бензиновая MS-4516, шина 40 см, 45...",Силовое оборудование/Пилы цепные/Бензопилы,Силовое оборудование,Пилы цепные,Бензопилы
14430,"Пила цепная бензиновая MS-5820, шина 50 см, 58...",Силовое оборудование/Пилы цепные/Бензопилы,Силовое оборудование,Пилы цепные,Бензопилы
14431,"Кусторез аккумуляторный HTD-510, нож 510 мм, L...",Садовый инвентарь/Ножницы садовые/Садовые ножн...,Садовый инвентарь,Ножницы садовые,Садовые ножницы и кусторезы аккумуляторные


In [157]:
train_data['Main_category'] = train_data['Section'].apply(lambda x: get_category_part_by_index(x,0))

In [158]:
train_data['Child_category'] = train_data['Section'].apply(lambda x: get_category_part_by_index(x,1))
train_data['Product_type'] = train_data['Section'].apply(lambda x: get_category_part_by_index(x,2))


In [224]:
from difflib import SequenceMatcher

# Функция для поиска синонимов
def find_synonyms(df1, df2, column):
    synonyms = {}
    for type1 in df1[column].unique():
        for type2 in df2[column].unique():
            if SequenceMatcher(None, type1, type2).ratio() > 0.6:  # Порог сходства
                synonyms[type1] = type2
                break
    return synonyms


In [225]:
unique_cats = train_data[['Main_category','Child_category','Product_type']]

In [226]:
unique_cats = unique_cats.drop_duplicates(subset='Product_type')

In [227]:
synonyms = find_synonyms(train_data, test_data, 'Product_type')

In [228]:
test_data

,Name,Product_type
0,"Биты с торцевыми головками 10 мм, 45 мм, 2 шт....",Бита
1,"Биты с торцевыми головками 8 мм, 45 мм, 2 шт. ...",Бита
2,"Биты с торцевыми головками 13 мм, 65 мм, 2 шт....",Бита
3,"Биты с торцевыми головками 8 мм, 65 мм, 2 шт. ...",Бита
4,"Биты с торцевыми головками 8 мм, 45 мм, 2 шт. ...",Бита
...,...,...
2885,"Нить для герметизации резьбовых соединений, ун...",Герметизирующая лента для кабеля
2886,"Нить для герметизации резьбовых соединений, ун...",Герметизирующая лента для кабеля
2887,"Нить для герметизации резьбовых соединений, ун...",Герметизирующая лента для кабеля
2888,"Нить для герметизации резьбовых соединений, ун...",Герметизирующая лента для кабеля


In [229]:
result = pd.merge(test_data,train_data,on='Name', how='inner')


In [230]:
result

,Name,Product_type_x,Section,Main_category,Child_category,Product_type_y
0,"Скобы, 6 мм, для мебельного степлера, закаленн...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
1,"Скобы, 8 мм, для мебельного степлера, закаленн...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
2,"Скобы, 14 мм, для мебельного степлера, тип 53,...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
3,"Скобы, 14 мм, для кабеля, закаленные, для степ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
4,"Скобы, 14 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
5,"Скобы, 12 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
6,"Скобы, 10 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
7,"Скобы, 12 мм, для кабеля, закаленные, для степ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
8,"Скобы, 8 мм, для мебельного степлера, тип 53, ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров
9,"Скобы, 6 мм, для мебельного степлера, тип 53, ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для мебельных степлеров


In [231]:
result = result.rename(columns={'Product_type_x':'Product_type','Product_type_y': 'Predicted_product_type'})

In [232]:
result['Predicted_product_type'] = result['Predicted_product_type'].replace(synonyms)

In [233]:
result

,Name,Product_type,Section,Main_category,Child_category,Predicted_product_type
0,"Скобы, 6 мм, для мебельного степлера, закаленн...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
1,"Скобы, 8 мм, для мебельного степлера, закаленн...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
2,"Скобы, 14 мм, для мебельного степлера, тип 53,...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
3,"Скобы, 14 мм, для кабеля, закаленные, для степ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
4,"Скобы, 14 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
5,"Скобы, 12 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
6,"Скобы, 10 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
7,"Скобы, 12 мм, для кабеля, закаленные, для степ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
8,"Скобы, 8 мм, для мебельного степлера, тип 53, ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера
9,"Скобы, 6 мм, для мебельного степлера, тип 53, ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера


In [234]:
result['Is_correctly_predicted'] = result['Product_type'] == result['Predicted_product_type']

In [235]:
result

,Name,Product_type,Section,Main_category,Child_category,Predicted_product_type,Is_correctly_predicted
0,"Скобы, 6 мм, для мебельного степлера, закаленн...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
1,"Скобы, 8 мм, для мебельного степлера, закаленн...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
2,"Скобы, 14 мм, для мебельного степлера, тип 53,...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
3,"Скобы, 14 мм, для кабеля, закаленные, для степ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
4,"Скобы, 14 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
5,"Скобы, 12 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
6,"Скобы, 10 мм, для мебельного степлера, закален...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
7,"Скобы, 12 мм, для кабеля, закаленные, для степ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
8,"Скобы, 8 мм, для мебельного степлера, тип 53, ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True
9,"Скобы, 6 мм, для мебельного степлера, тип 53, ...",Скобы для строительного степлера,Крепежный инструмент/Скобы для мебельных степл...,Крепежный инструмент,,Скобы для строительного степлера,True


In [236]:
accuracy = result['Is_correctly_predicted'].apply(lambda x: 1 if x else 0).sum() / result.Is_correctly_predicted.count()

In [237]:
accuracy

np.float64(0.6)